In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [50]:
average_movie_train_df = pd.DataFrame({'rating_avg': data_train.groupby('movie_id')['rating'].mean()}).reset_index()
average_movie_train_df.sort_values(by='rating_avg', ascending=False, inplace=True)
average_movie_train_df = average_movie_train_df.reset_index(drop=True)
user_movies_train_df = pd.DataFrame({'movies_list': data_train.groupby('user_id')['movie_id'].apply(list)}).reset_index()
user_movies_test_df = pd.DataFrame({'movies_list': data_test.groupby('user_id')['movie_id'].apply(list)}).reset_index()

train_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data_train = pd.read_csv("ml-100k/u1.base", delimiter='\t', names=train_cols)
test_cols = ['user_id', 'movie_id', 'rating', 'timestamp', 'predict_rating', 'difference']
data_test = pd.read_csv("ml-100k/u1.test", delimiter='\t', names=test_cols)
average_movie_train = data_train.groupby('movie_id')['rating'].mean()
average_movie_train_int = average_movie_train.astype(int)
avg = average_movie_train_int.values.mean()
avg = avg.astype(int)

In [51]:
def user_recommendations():
    user_recommendations_avg = {}
    user_recommendations_random = {}
    for index, row in user_movies_train_df.iterrows():
        id = row['user_id']
        movies = row['movies_list']
        rec_list_avg = []
        rec_list_random = []
        i = 0
        # Average
        while len(rec_list_avg) < 20:
            must_pop_avg = average_movie_train_df.iloc[i]['movie_id'].astype(int)        
            if must_pop_avg not in movies:
                rec_list_avg.append(must_pop_avg)
            i = i + 1
        user_recommendations_avg[id] = rec_list_avg
        # Random
        rec_list_random = list(average_movie_train_df.sample(n=20)['movie_id'])
        user_recommendations_random[id] = rec_list_random
    # print(user_recommendations_avg)
    # print(user_recommendations_random)
    
    
user_recommendations()

In [52]:
def avg_movie(movie_id):
    if movie_id in average_movie_train_int:
        return average_movie_train_int[movie_id]
    else:
        return None


def calc_mae():
    global data_test
    for index, row in data_test.iterrows():
        predict_value = avg_movie(row['movie_id'])
        if predict_value is None:
            predict_value = avg
        data_test.at[index, 'predict_rating'] = predict_value    
        real_value = row['rating']
        difference = pd.np.absolute(real_value - predict_value)
        data_test.at[index, 'difference'] = difference 
    numerator = data_test['difference'].sum()
    denominator = data_test.shape[0]
    mae = numerator/denominator
    print(mae)

        
calc_mae()



0.94175


In [53]:
def recall_precision():
    user_recall_avg = []
    user_recall_random = []
    user_precision_avg = []
    user_precision_random = []
    
    for index, row in user_movies_test_df.iterrows():
        id = row['user_id']
        true = row['movies_list']
        pred_avg = user_recommendations_avg[id]
        pred_random = user_recommendations_random[id]
        
        size_true = len(true)
        size_pred_avg = len(pred_avg)    
        size_pred_random = len(pred_random)
        
        common_avg = list(set(true).intersection(pred_avg))
        common_random = list(set(true).intersection(pred_random))
        
        number_common_avg = len(common_avg)
        number_common_random = len(common_random)
        
        recall_avg = number_common_avg / size_true
        user_recall_avg.append(recall_avg)
        recall_random = number_common_random / size_true
        user_recall_random.append(recall_random)
        
        precision_avg = number_common_avg / size_pred_avg
        user_precision_avg.append(precision_avg)
        precision_random = number_common_random / size_pred_random
        user_precision_random.append(precision_random)
    
    recall_for_avg = np.mean(user_recall_avg)
    recall_for_random = np.mean(user_recall_random)
    precision_for_avg = np.mean(user_precision_avg)
    precision_for_random = np.mean(user_precision_random)
    print(recall_for_avg)
    print(recall_for_random)
    print(precision_for_avg)
    print(precision_for_random)


recall_precision()

0.02233534107339741
0.010657634163740392
0.04586056644880174
0.025816993464052286


In [82]:
average_movie_train_df = pd.DataFrame({'rating_avg': data_train.groupby('movie_id')['rating'].mean()}).reset_index()
average_movie_train_df.sort_values(by='rating_avg', ascending=False, inplace=True)
average_movie_train_df = average_movie_train_df.reset_index(drop=True)
user_movies_train_df = pd.DataFrame({'movies_list': data_train.groupby('user_id')['movie_id'].apply(list)}).reset_index()
user_movies_test_df = pd.DataFrame({'movies_list': data_test.groupby('user_id')['movie_id'].apply(list)}).reset_index()

train_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data_train = pd.read_csv("ml-100k/u1.base", delimiter='\t', names=train_cols)
test_cols = ['user_id', 'movie_id', 'rating', 'timestamp', 'predict_rating', 'difference']
data_test = pd.read_csv("ml-100k/u1.test", delimiter='\t', names=test_cols)
average_movie_train = data_train.groupby('movie_id')['rating'].mean()
average_movie_train_int = average_movie_train.astype(int)
avg = average_movie_train_int.values.mean()
avg = avg.astype(int)